In [0]:
prec := 200;
 Q := RationalField();
 P<x> := PolynomialRing(Rationals());
 f:= x^4 - x^3 - 5*x^2 + 2*x + 1 ;
R<s> := NumberField(f); //change the totally real quartic field here

 PP<x> := PolynomialRing(R);
 RF<im> := NumberField(PP![1,0,1]); //change the integer in "CM by sqrt(-integer)" here

 CMF<a> := AbsoluteField(RF);

/*generating a principal polarization xi when class number is 1*/
O := MaximalOrder(CMF);
D:= Different(O);
Oo := Integers(R);
Dreal := Different(Oo);
_,xireal := IsPrincipal(Dreal^-1);
xi := CMF!((CMF ! xireal)*im);
_,xi0 := IsPrincipal(D^-1);
AbsoluteNorm(xi/xi0);


/*since 2 is the problem...*/
I := ideal<Oo|2>;
twofactor := 2;
deg := [1,2,3,4];
if Norm(xi/(twofactor*xi0)) ne 1 and Norm(xi/xi0) ne 1 then
    for Ii in Factorization(I) do
        _,a := IsPrincipal(Ii[1]);
        for d in deg do
            if Norm(xi/((a^d)*xi0)) eq 1 then
                twofactor := a^d;
                printf("2 factor fixed");
                break;
            end if;
        end for;
    end for;
    elif Norm(xi/xi0) eq 1 then
        twofactor := 1;
end if;
xi := xi/twofactor;
xireal := xireal/twofactor;
Norm(xi/xi0);



In [0]:
/*On the look out if Shimura class group is not trivial*/
UR, f := UnitGroup(R);
unitgen := [R!f(g): g in Generators(UR)];
Append(~unitgen, 1);
primunit := [* *];
for i,j,k in [0,1] do
    u := (unitgen[1]^i) * (unitgen[2]^j) * (unitgen[3]^k);
    Append(~primunit, u);
end for;


//[[Re(Conjugate(u, i)) gt 0: i in deg]: u in primunit];
twistingunit := [* *];
for u in primunit do 
    if IsTotallyPositive(u) or IsTotallyPositive(-u) then
        Append(~twistingunit, u);
    end if;
end for;

if #twistingunit gt 1 then
    printf("There are more than one xi, use twisingunit list to generate more polarizations");
else printf("Lucky you, there's only one possible polarization");
end if;
#twistingunit;

/*Weil type generator revamped*/

RePla := RealPlaces(R);

function cmcounter(xi)
counter := 0;
deg := [1,2,3,4];
for i in deg do
    if Re(Conjugate(xi, i)) gt 0 then
    counter := counter+1;
    end if;
end for;
return counter;
end function;

cmcounter(xireal);

possiblexi := [* *];
for u in primunit do 
    if cmcounter(u*xireal) eq 2 then
        Append(~possiblexi, xi*u);
    end if;
end for;
#possiblexi;


In [0]:

for xi in possiblexi do

cc := Automorphisms(CMF)[2];

/*Generating period matrix*/

Z := IntegerRing();
E := Matrix(Z, [[Trace(xi*cc(a)*b) : b in Basis(O)] : a in Basis(O)]);

D, C := FrobeniusFormAlternating(E);

g := Degree(CMF) div 2;

newb := ElementToSequence(Matrix(O,C)*Matrix(O,2*g,1,Basis(O)));

conj := [i  : i in [1..2*g] | Abs(Re(Conjugate(xi,i))) lt 10^-10 and Im(Conjugate(xi,i)) gt 0];
BigPM := Matrix([[Conjugate(b,i : Precision:=prec) : b in newb] : i in conj]);
tau := Submatrix(BigPM, 1, g + 1, g, g)^-1*Submatrix(BigPM, 1, 1, g, g);



function IntegerReduceMatrixG4(tau);
tauZZ := Matrix([ [ Round(Re(c)) : c in Eltseq(row) ] : row in Rows(tau) ]);
tauZZ[2,1] := tauZZ[1,2];
tauZZ[3,1] := tauZZ[1,3];
tauZZ[3,2] := tauZZ[2,3];
tauZZ[4,1] := tauZZ[1,4];
tauZZ[4,2] := tauZZ[2,4];
tauZZ[4,3] := tauZZ[3,4];
return tau - ChangeRing(tauZZ, BaseRing(tau));
end function;

function LLLReduceMatrixG4(tau);
//assert IsSmallPeriodMatrix(tau);
tau[2,1] := tau[1,2];
tau[3,1] := tau[1,3];
tau[3,2] := tau[2,3];
tau[4,1] := tau[1,4];
tau[4,2] := tau[2,4];
tau[4,3] := tau[3,4];
Imtau := Matrix([ [ Im(c) : c in Eltseq(row) ] : row in Rows(tau) ]);
_, T := LLLGram(Imtau);
return T*tau*Transpose(T);
end function;

function LeftActionHg(M, tau)
//assert IsSmallPeriodMatrix(tau);
g := #Rows(tau);
A := Submatrix(M, 1,  1, g,g); B := Submatrix(M, 1,  g+1, g,g);
C := Submatrix(M, g+1,1, g,g); D := Submatrix(M, g+1,g+1, g,g);
return (A*tau + B)*((C*tau + D)^(-1));
end function;

function ReduceSmallPeriodMatrixG4(tau)
N0 := Matrix(Integers(), [
[  0,  0,  0,  0,  1,  0,  0,  0],
[  0,  1,  0,  0,  0,  0,  0,  0],
[  0,  0,  1,  0,  0,  0,  0,  0],
[  0,  0,  0,  1,  0,  0,  0,  0],
[ -1,  0,  0,  0,  0,  0,  0,  0],
[  0,  0,  0,  0,  0,  1,  0,  0],
[  0,  0,  0,  0,  0,  0,  1,  0],
[  0,  0,  0,  0,  0,  0,  0,  1]
]);
counter := 0;
while true do
    counter +:= 1;
    /*
    if counter mod 10^3 eq 0 then
        vprint CurveRec : "";
        vprint CurveRec : "Counter for period matrix reduction is high:", counter;
        break;
    end if;
    */

    tau := LLLReduceMatrixG4(tau);



    tau := IntegerReduceMatrixG4(tau);


    if Abs(tau[1,1]) gt 1 then
        return tau;
    end if;
    tau := LeftActionHg(N0, tau);
    
end while;
end function;

tau := ReduceSmallPeriodMatrixG4(tau);
printf("Absolute value of period matrix");
Abs(Determinant(tau));

/*using magma to compute the Schottky form*/
 C := ComplexField(200);
char := Matrix(C, 8, 1, [0,0,0,0,0,0,0,0]);
z := Matrix(C, 4, 1, [0,0,0,0]);

m1 := 1/2*Matrix(C, 8, 1, [1,0,1,0,1,0,1,0]);
m2 := 1/2*Matrix(C, 8, 1, [0,0,0,1,1,0,0,0]);
m3 := 1/2*Matrix(C, 8, 1, [0,0,1,1,1,0,1,1]);
n0 := Matrix(C, 8, 1, [0,0,0,0,0,0,0,0]);
n1 := 1/2*Matrix(C, 8, 1, [0,0,0,1,1,1,1,0]);
n2 := 1/2*Matrix(C, 8, 1, [0,0,1,1,0,0,0,1]);
n3 := 1/2*Matrix(C, 8, 1, [0,0,1,0,1,0,1,1]);
n4 := n1+n2;
n5 := n1+n3;
n6 := n2+n3;
n7 := n1+n2+n3;
SchottkyN := [n0,n1,n2,n3,n4,n5,n6,n7];
M1 := [m1 + n: n in SchottkyN];
M2 := [m2 + n: n in SchottkyN];
M3 := [m3 + n: n in SchottkyN];
pi1 := 1;
pi2 := 1;
pi3 := 1;
for m in M1 do
    pi1 := pi1* Theta(m, z, tau);
end for;

for m in M2 do
    pi2 := pi2* Theta(m, z, tau);
end for;

for m in M3 do
    pi3 := pi3* Theta(m, z, tau);
end for;

Schottky := pi1^2 + pi2^2 + pi3^2 - 2* (pi1*pi2 + pi2*pi3 + pi1*pi3);
printf("Absolute value of Schottky-Igusa form");
Abs(Schottky);
end for;

In [0]:
//fields that need some fractional ideal to do the job: x^4 - 2*x^3 - 4*x^2 + 3*x + 3

In [0]:
//trace problem x^4 - x^3 - 5*x^2 + 3*x + 4 